In [ ]:
from keras.applications.resnet import ResNet50,preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import *
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.models import Sequential,Model,load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
import numpy as npand 
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras.utils import image_utils

In [ ]:
height=400
width=400
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

In [ ]:
train_dir="data/train_img"
test_dir="data/test_img"

batch_size=8

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(height,width),batch_size=batch_size)


validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
validation_generator=validation_datagen.flow_from_directory(test_dir,target_size=(height,width),batch_size=batch_size)

In [ ]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)

    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)

    predictions = Dense(num_classes, activation='softmax')(x)
    finetune_model = Model(inputs=base_model.input, outputs=predictions)
    return finetune_model


class_list = ['genuine', 'counterfit']
FC_Layers = [1024, 1024]
dropout = 0.5
finetune_model = build_finetune_model(
    base_model, dropout=dropout, fc_layers=FC_Layers, num_classes=len(class_list))


In [ ]:
num_epochs=100
num_train_images=134

In [ ]:
checkpoint = ModelCheckpoint(
    "model.h5",
    monitor="val_acc",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    period=1,
)
early = EarlyStopping(
    monitor="val_acc", min_delta=0, patience=40, verbose=1, mode="auto"
)


finetune_model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.SGD(lr=0.000001, momentum=0.9),
    metrics=["accuracy"],
)

# train
history = finetune_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=num_train_images // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=1,
    callbacks=[checkpoint, early],
)

# save
# finetune_model.save_weights("model\\model.h5")


In [ ]:
finetune_model.save("model\\model.h5")

In [ ]:
# If model is already trained
model = keras.models.load_model(
        "model\\model.h5"
    )